In [58]:
import sys, os, glob
import torch
import torch.nn as nn
import torch.nn.functional as F

from transformers import (
    AutoModel,
    AutoModelForSequenceClassification,
    AutoModelForMaskedLM,
    AutoTokenizer
)
from transformers.models.bert.configuration_bert import BertConfig
from DGXutils import GetFileNames, GetLowestGPU
from importlib import reload

sys.path.append('../')

import utils.GPTransformer as GPTransformer

In [59]:
device = GetLowestGPU()
model_path = 'zhangtaolab/dnabert2-conservation'

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(f"{model_path}", trust_remote_code=True)

In [60]:
reload(GPTransformer)

# load model
model = GPTransformer.GPTransformer(model_path,
                                    n_mlp=0)
model = model.to(device)

/home/pkr/.cache/huggingface/modules/transformers_modules/zhangtaolab/dnabert2-conservation/5babb4ae2a106150ab00cb4487d95f83293b9dc6/bert_layers.py:126: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(


In [61]:
# load hf model
config = BertConfig.from_pretrained(f"{model_path}")
hf_model = AutoModelForSequenceClassification.from_pretrained(f'{model_path}', 
                                                                    trust_remote_code=True,
                                                                    config=config).to(device)

In [62]:
dna = "ACGTAGCATCGGATCTATCTATCGACACTTGGTTATCGATCTACGAGCATCTCGTTAGC"
inputs = tokenizer(dna, return_tensors="pt").to(device)
print(inputs['input_ids'])

tensor([[   1,    5,  194,   32,  757, 1239, 2092,  294,   24,  359,   88,   93,
           32,   75,   77,   19,    2]], device='cuda:3')


In [63]:
pheno = model(inputs['input_ids'], inputs['attention_mask'])

In [64]:
pheno.shape

torch.Size([1, 4])

In [65]:
model

GPTransformer(
  (hf_model): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(4096, 768, padding_idx=3)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertUnpadAttention(
              (self): BertUnpadSelfAttention(
                (dropout): Dropout(p=0.0, inplace=False)
                (Wqkv): Linear(in_features=768, out_features=2304, bias=True)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
            )
            